# Quantinuum Nexus Quick Start





<div class="alert alert-block alert-info"><b>NB:</b> If you will be using Quantinuum Nexus for this hackathon, please follow the 'Quantinuum Nexus Instructions' in the README.md before following along with this notebook.</div>

## Welcome to the Quantinuum Nexus Lab! A hosted Jupyter environment with the pre-installed Python software packages you'll need, including:

* `pytket`: Quantinuum's Python SDK for TKET quantum programming tools.
* `qnexus`: the client software for submitting jobs to the Quantinuum Nexus platform

### Step 1. Enable access to the Quantinuum simulator and emulator

We provide a number of systems for you to target when executing quantum circuits as part of the challenge.

* `H1-1LE`: a noiseless simulator of Quantinuum's 20 qubit `H1-1` quantum computer.
* `H1-Emulator` and `H2-Emulator`: simulators with full error models for emulating Quantinuum Systems quantum hardware.


### Step 2: Try out the example code below to compile and simulate your first circuit.

- Copy this file or its contents into the Quantinuum Nexus Lab environment and run the code below to see some example usage.


## Lets compose a quantum circuit using Pytket

Nexus uses [pytket Circuits](https://cqcl.github.io/pytket/manual/manual_circuit.html). Here we'll create a simple one.

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

my_circuit = Circuit(2,2).H(0).CX(0, 1).measure_all()

render_circuit_jupyter(my_circuit)

## Set up `qnexus`

The qnexus package allows you to interact with the Quantinuum Nexus cloud.

In [ ]:
import qnexus as qnx

# Then we'll create a project that will be used to contain our jobs and data

my_project_ref = qnx.projects.get_or_create(name="My First Nexus Project")

You should be able to see this project if you visit [https://nexus.quantinuum.com/projects](https://nexus.quantinuum.com/projects)


We can now upload our circuit to Nexus.

In [ ]:
my_circuit_ref = qnx.circuits.upload(
    circuit=my_circuit,
    project=my_project_ref,
    name="My Bell State Circuit",
    description="This is my first circuit uploaded to Nexus",
)

### Set up your Backend Configuration

In Quantinuum Nexus, we use `BackendConfig`s to specify the details of the quantum system we want to target.


For this example, we'll use ``QuantinuumConfig``, to configure Nexus to target a simulator for Quantinuum's `H1-Emulator` device. 


In [ ]:
configuration = qnx.QuantinuumConfig(device_name="H1-Emulator")

### Compile the circuit

Nexus offers compilation with a cloud-hosted version of the TKET quantum compiler, this is an easy way to:

* Optimize the circuit
* Convert it to run on a specific quantum device, which might not support all quantum gates.

If you'd like to disable any changes to your circuit other than making sure that the gateset is correct, you can always set `optimisation_level=0`.

In [ ]:
ref_compile_job = qnx.start_compile_job(
    circuits=[my_circuit_ref],
    backend_config=configuration,
    optimisation_level=2,
    name="Bell State Compilation job",
    project=my_project_ref,
)

In [ ]:
qnx.jobs.wait_for(ref_compile_job)
ref_compiled_circuit = qnx.jobs.results(ref_compile_job)[0].get_output()

compiled_circuit = ref_compiled_circuit.download_circuit()

render_circuit_jupyter(compiled_circuit)

You should now see the `Compile Job` show up when you click on your project on [https://nexus.quantinuum.com](https://nexus.quantinuum.com).

If you're curious to see what the compiler did, you can click on the job results and see all the passes the compiler performed on your circuit. 

### Submit the circuit for execution

Nexus will take care of submitting the circuit to your chosen device and storing the results when it has executed.

In this case we are executing the circuit on a simulator running in the Nexus cluster so we should get the results back quickly, however many quantum devices have lengthy queues in which case Nexus will track the job and retrieve the results when they are ready.

In [ ]:
ref_execute_job = qnx.start_execute_job(
    circuits=[ref_compiled_circuit],
    n_shots=[100],
    backend_config=configuration,
    name="Bell State Execution Job",
    project=my_project_ref,
)

In [ ]:
qnx.jobs.wait_for(ref_execute_job)
ref_result = qnx.jobs.results(ref_execute_job)[0]
backend_result = ref_result.download_result()

backend_result.get_distribution()

You can also view the job from the project page on the Quantinuum Nexus website and check out the results there.


For a more detailed version of this notebook please check out [https://docs.quantinuum.com/nexus/trainings/notebooks/basics/getting_started.html](https://docs.quantinuum.com/nexus/trainings/notebooks/basics/getting_started.html)


### Good luck with the challenge!